### Objetivo: Encontrar regiões mais adensadas e maior renda per capita em São Paulo ou outros municípios a fim de buscar maior quantidade de corridas premium como motorista de aplicativo.

A demanda por corridas premium em aplicativos como Uber e 99 pode ser influenciada por dois fatores principais: a densidade populacional e a renda per capita. É possível supor que, em áreas mais densamente povoadas, haja uma maior quantidade de pessoas se deslocando com frequência, o que tende a gerar mais solicitações de transporte por aplicativo. Nessas regiões, como em centros urbanos e bairros com alta concentração de habitantes, a lógica sugere que o uso desse tipo de serviço seja mais comum devido ao grande número de potenciais usuários.

Por outro lado, em locais onde a renda per capita é mais elevada, é razoável imaginar que o interesse por categorias premium, como Uber Black, seja maior. Isso se baseia na suposição de que pessoas com maior poder aquisitivo podem preferir serviços que ofereçam mais conforto e exclusividade, mesmo com tarifas mais altas. Assim, serviços que oferecem veículos de qualidade superior e motoristas com melhores qualificações podem atrair um público mais exigente, disposto a pagar por essa experiência.

Da mesma forma, categorias semi-premium, como Uber Comfort e 99 Plus, poderiam ser mais populares em regiões de alta renda, pois oferecem uma combinação de conforto a um custo relativamente mais acessível. Com base nisso, regiões de maior densidade populacional e renda mais alta poderiam ter uma demanda proporcionalmente maior por essas corridas premium e semi-premium, refletindo o perfil dos habitantes que vivem ou trabalham nessas localidades.

Em relação aos dados, as informações fornecidas pelo Censo serão essenciais para a descoberta das regiões mais adensadas e de maior renda per capita.

    Hoje, nós estamos em 2024 e o censo de 2022 saiu preliminarmente contendo apenas os dados populacionais. No entanto, já que o estudo envolve população e renda, será utilizado os dados de 2010 que estão completos. Sabe-se que em 14 anos, teremos diferença nos dados, mas ao analisar o aumento populacional em São Paulo e São Bernardo do Campo, 1,7% e 5,9% respectivamente, de 2010 a 2022, provavelmente, não obtém-se grandes diferenças habitacionais no período. No entanto, a atualização deste estudo se torna peremptória para a assertividade do objetivo estipulado assim que divulgado os dados do Censo de 2022.

    O que são os setores censitários?

        "Os setores censitários são a menor porção de área utilizadas pelo IBGE para planejar, coletar e disseminar os resultados dos Censos e Pesquisas Estatísticas. 
         Esses setores são definidos a partir de diversos critérios:
            * A área e o número de domicílios do setor devem ser passíveis de serem percorridos e visitados por um único recenseador em tempo hábil.
            * O recorte do setor deve ser representativo em relação à população local.
            * O setor deve ter um número de domicílios suficiente para que seus dados agregados preservem o sigilo dos moradores"
            (G1 - https://g1.globo.com/economia/censo/noticia/2024/03/21/o-que-e-o-setor-censitario-medida-usada-pelo-ibge-para-fazer-o-censo.ghtml)

    Dados:
    
        Diretório do censo de todos os estados do Brasil por setor censitário:
            https://ftp.ibge.gov.br/Censos/Censo_Demografico_2010/Resultados_do_Universo/Agregados_por_Setores_Censitarios/
    
        Diretório dos arquivos geográficos dos setores censitários:
            https://geoftp.ibge.gov.br/organizacao_do_territorio/malhas_territoriais/malhas_de_setores_censitarios__divisoes_intramunicipais/censo_2010/setores_censitarios_shp/

---

## Importação de bibliotecas

In [95]:
import os # Criação de estrutura de pastas e manuseamento de arquivos
from ftplib import FTP # Biblioteca para download dos arquivos de servidor ftp
from zipfile import ZipFile # Biblioteca para descompactar os arquivos
import pandas as pd # Tratamento dos dados 
import geopandas as gpd # Tratamento dos dados geográficos
import matplotlib.pyplot as plt # Biblioteca para visualização gráfica dos dados
import folium # Biblioteca para visualização em mapas

---

## Estrutura de pastas

In [96]:
parent_dir = os.getcwd() # Identificação do diretório de trabalho

"""
Criação dos diretórios para arquivamento dos arquivos
data: Arquivos baixados e a serem utilizados para tratamento.
artifacts: Todos os arquivos criados para utilização e visualização.
"""

directories = ["data", "artifacts"] 
for directory in directories:
    path = os.path.join(parent_dir, directory)
    try:
        os.mkdir(path)
    except FileExistsError:
        print("Directory already created!")
        
data_dir = os.path.join(parent_dir, directories[0])
artifacts_dir = os.path.join(parent_dir, directories[1])

Directory already created!
Directory already created!


### Download de arquivos

Acessando os servidores FTP e baixando os arquivos.

In [97]:
ftp_server = "ftp.ibge.gov.br" # Servidor FTP do IBGE
ftp_dir = "Censos/Censo_Demografico_2010/Resultados_do_Universo/"\
    "Agregados_por_Setores_Censitarios/" # Diretório a ser acessado
# O servidor aceita acesso público, portanto user e password desnecessários
user = "anonymous" # Inserir anonymous para entrar no servidor
passwd = ""

ftp = FTP(ftp_server, user, passwd) # Configuração da variável de acesso ao servidor
ftp.cwd(ftp_dir) # Acessando o diretório escolhido
files = ftp.nlst() # variável contendo o conteúdo do diretório

ftp.dir() # Listagem do conteúdo do diretório

-rwxrwxr-x    1 ftp      ftp          1979 Oct 30  2023 1_Atualizacoes_20231030.txt
-rwxrwxr-x    1 ftp      ftp       5579523 Oct 30  2023 AC_20231030.zip
-rwxrwxr-x    1 ftp      ftp      22399914 Oct 30  2023 AL_20231030.zip
-rwxrwxr-x    1 ftp      ftp      31064700 Oct 30  2023 AM_20231030.zip
-rwxrwxr-x    1 ftp      ftp       5194028 Oct 30  2023 AP_20231030.zip
-rwxrwxr-x    1 ftp      ftp      126886126 Oct 30  2023 BA_20231030.zip
-rwxrwxr-x    1 ftp      ftp      71781739 Oct 30  2023 CE_20231030.zip
-rwxrwxr-x    1 ftp      ftp      21585996 Oct 30  2023 DF_20231030.zip
-rwxrwxr-x    1 ftp      ftp      34133643 Oct 30  2023 Documentacao_Agregado_dos_Setores_2010_20231030.zip
-rwxrwxr-x    1 ftp      ftp      37084676 Oct 30  2023 ES_20231030.zip
-rwxrwxr-x    1 ftp      ftp      53664403 Oct 30  2023 GO_20231030.zip
-rwxrwxr-x    1 ftp      ftp      48421505 Oct 30  2023 MA_20231030.zip
-rwxrwxr-x    1 ftp      ftp      169948641 Oct 30  2023 MG_20231030.zip
-rwxrwxr-x    

Nota-se que temos 3 tipos de arquivos:
  * 1 arquivo texto
  * 1 arquivo de documentação compactado
  * Arquivos do censo de cada estado compactado

In [98]:
docum = "Documentacao_Agregado" # variável para obter o arquivo de documentação
UFs = ["SP"] # lsita dos estados desejados
files_check = [ filename for filename in os.listdir(data_dir) ] # lista dos arquivos no diretório 

# Loop para download dos arquivos no servidor FTP
for file in files:
        if (file.startswith(docum) or file[:2] in UFs) and (file not in files_check):
            with open(os.path.join(data_dir,file), 'wb') as fp:
                ftp.retrbinary('RETR ' + file, fp.write)
                print(file+" downloaded to "+data_dir)

ftp.quit() # fechamento da conexão com o servidor FTP

'221 Goodbye.'

In [99]:
# Acessando o servidor ftp com as informações geográficas dos setores censitários
ftp_server = "geoftp.ibge.gov.br"
ftp_dir = "organizacao_do_territorio/malhas_territoriais/"\
    "malhas_de_setores_censitarios__divisoes_intramunicipais/censo_2010/"\
    "setores_censitarios_shp/"
user = "anonymous"
passwd = ""

ftp = FTP(ftp_server, user, passwd)
ftp.cwd(ftp_dir)
ftp.dir()

drwxrwxr-x    2 ftp      ftp          4096 Jun 14  2016 1_leia_me
drwxrwxr-x    2 ftp      ftp          4096 Jun 14  2016 ac
drwxrwxr-x    2 ftp      ftp          4096 Jun 14  2016 al
drwxrwxr-x    2 ftp      ftp          4096 Jun 14  2016 am
drwxrwxr-x    2 ftp      ftp          4096 Jun 14  2016 ap
drwxrwxr-x    2 ftp      ftp          4096 Jun 14  2016 ba
drwxrwxr-x    2 ftp      ftp          4096 Jun 14  2016 ce
drwxrwxr-x    2 ftp      ftp          4096 Jun 14  2016 df
drwxrwxr-x    2 ftp      ftp          4096 Jun 14  2016 es
drwxrwxr-x    2 ftp      ftp          4096 Jun 14  2016 go
drwxrwxr-x    2 ftp      ftp          4096 Jun 14  2016 ma
drwxrwxr-x    2 ftp      ftp          4096 Jun 14  2016 mg
drwxrwxr-x    2 ftp      ftp          4096 Jun 14  2016 ms
drwxrwxr-x    2 ftp      ftp          4096 Jun 14  2016 mt
drwxrwxr-x    2 ftp      ftp          4096 Jun 14  2016 pa
drwxrwxr-x    2 ftp      ftp          4096 Jun 14  2016 pb
drwxrwxr-x    2 ftp      ftp          4096 Jun 14

In [100]:
ftp.dir('sp')

-rw-rw-r--    1 ftp      ftp      11116004 Jun 14  2016 sp_distritos.zip
-rw-rw-r--    1 ftp      ftp       9387313 Jun 14  2016 sp_municipios.zip
-rwxrwxr-x    1 ftp      ftp      45523805 Nov 04  2020 sp_setores_censitarios.zip
-rw-rw-r--    1 ftp      ftp      11108462 Jun 14  2016 sp_subdistritos.zip


Verifica-se que dentro do diretório há quatro arquivos com as divisões dos municípios, distritos, subdistritos e setores censitários. Neste momento, deseja-se os setores censitários.

In [101]:
UFs_geo = ['sp'] # lista dos estados
extension_geo = "_setores_censitarios.zip" # variável com a extensão dos arquivos a serem baixados
files_check = [ filename for filename in os.listdir(data_dir) ] 

# Loop entre todos os diretórios dos estados para download dos arquivos geográficos
for UF_geo in UFs_geo:
    ftp.cwd(UF_geo) 
    files = ftp.nlst()
    for file in files:
        if (file == UF_geo+extension_geo) and (file not in files_check):
            with open(os.path.join(data_dir,file), 'wb') as fp:
                ftp.retrbinary('RETR ' + file, fp.write)
                print(file+" downloaded to "+data_dir)
    ftp.cwd("..") # Voltando ao diretório anterior para repetição do processo
ftp.quit()

'221 Goodbye.'

---

## Extração dos arquivos compactados.

In [102]:
data_dir_files = os.listdir(data_dir) # Lista dos arquivos no diretório 'data'
data_dir_files

['Documentacao_Agregado_dos_Setores_2010_20231030',
 'Documentacao_Agregado_dos_Setores_2010_20231030.zip',
 'SP_Capital_20231030',
 'SP_Capital_20231030.zip',
 'SP_Exceto_Capital_20231030',
 'SP_Exceto_Capital_20231030.zip',
 'sp_setores_censitarios',
 'sp_setores_censitarios.zip']

In [103]:
# Loop para extração dos arquivos compactados
for file in data_dir_files:
    try:
        if file.split(".")[1] == "zip": # Verificação se são arquivos .zip
            file_zipped = ZipFile(os.path.join(data_dir,file))
            
            new_folder_path = os.path.join(data_dir, file.split(".zip")[0]) # Diretório para receber os arquivos
            
            # Criação do diretório caso não exista
            if not os.path.exists(new_folder_path):
                file_zipped.extractall(new_folder_path)
            
            file_zipped.close()
    except:
        print(f"{file} - Not a zipped file")

Documentacao_Agregado_dos_Setores_2010_20231030 - Not a zipped file
SP_Capital_20231030 - Not a zipped file
SP_Exceto_Capital_20231030 - Not a zipped file
sp_setores_censitarios - Not a zipped file


In [104]:
files_list =[]

for dirpath, dirnames, filenames in os.walk(data_dir):
    for filename in filenames:
        file_path = os.path.join(dirpath, filename)
        files_list.append(file_path)
        
files_list

['c:\\Users\\guicr\\OneDrive\\Carreira\\Portfolio\\uber\\data\\Documentacao_Agregado_dos_Setores_2010_20231030.zip',
 'c:\\Users\\guicr\\OneDrive\\Carreira\\Portfolio\\uber\\data\\SP_Capital_20231030.zip',
 'c:\\Users\\guicr\\OneDrive\\Carreira\\Portfolio\\uber\\data\\SP_Exceto_Capital_20231030.zip',
 'c:\\Users\\guicr\\OneDrive\\Carreira\\Portfolio\\uber\\data\\sp_setores_censitarios.zip',
 'c:\\Users\\guicr\\OneDrive\\Carreira\\Portfolio\\uber\\data\\Documentacao_Agregado_dos_Setores_2010_20231030\\Documentaç╞o\\BASE DE INFORMAÇσES POR SETOR CENSIT╡RIO Censo 2010 - Universo novo.pdf',
 'c:\\Users\\guicr\\OneDrive\\Carreira\\Portfolio\\uber\\data\\Documentacao_Agregado_dos_Setores_2010_20231030\\Documentaç╞o\\Descriç╞o_AC.xls',
 'c:\\Users\\guicr\\OneDrive\\Carreira\\Portfolio\\uber\\data\\Documentacao_Agregado_dos_Setores_2010_20231030\\Documentaç╞o\\Descriç╞o_AL.xls',
 'c:\\Users\\guicr\\OneDrive\\Carreira\\Portfolio\\uber\\data\\Documentacao_Agregado_dos_Setores_2010_20231030\\Docu

Ao observar a lista de arquivos, percebe-se uma duplicidade entre os arquivos .csv e .xls referentes aos dados dos setores censitários. Então, podemos eliminar os arquivos duplicados que estão nas pastas com o formato: c:/Users/guicr/OneDrive/Carreira/Portfolio/uber/data/SP_Capital_20231030/Base informaçoes setores2010 universo SP_Capital/EXCEL/

In [105]:
# Diretórios a serem excluídos
mask = [os.path.join(data_dir, "SP_Capital_20231030\\Base informaçoes setores2010 universo SP_Capital\\EXCEL"),
        os.path.join(data_dir, "SP_Exceto_Capital_20231030\\Base informaçoes setores2010 universo SP_Exceto_Capital\\EXCEL")
        ]

# Loop para remoção dos arquivos e diretório
for item in mask:
    for dirpath, dirnames, filenames in os.walk(item, topdown=False):
        for filename in filenames:
                os.remove(os.path.join(dirpath, filename))
        os.rmdir(dirpath)

---

## Tratamento de dados

In [106]:
# Passando a lista de arquivos para um dataframe para utilização mais fácil dos arquivos
files_list_df =pd.DataFrame(files_list, columns=['filepaths'])
files_list_df

,filepaths
0,c:\Users\guicr\OneDrive\Carreira\Portfolio\ube...
1,c:\Users\guicr\OneDrive\Carreira\Portfolio\ube...
2,c:\Users\guicr\OneDrive\Carreira\Portfolio\ube...
3,c:\Users\guicr\OneDrive\Carreira\Portfolio\ube...
4,c:\Users\guicr\OneDrive\Carreira\Portfolio\ube...
...,...
84,c:\Users\guicr\OneDrive\Carreira\Portfolio\ube...
85,c:\Users\guicr\OneDrive\Carreira\Portfolio\ube...
86,c:\Users\guicr\OneDrive\Carreira\Portfolio\ube...
87,c:\Users\guicr\OneDrive\Carreira\Portfolio\ube...


Agora, será necessário utilizar os arquivos necessários do censo para retirar a população e renda de cada setor censitário, para isso serão utilizados dois arquivos de acordo com a documentação e eles possuem os seguintes formatos Basico_UF.csv e Pessoa13_UF (necessário trocar UF pelo estado a ser observado). No caso de São Paulo, existem dois tipos de arquivo um para a capital e outro para os demais municípios, os quais unir-se-ão adiante.

As informações de renda e população estão contidas no arquivo Basico_UF.csv e Pessoa13_UF, respectivamente.

In [107]:
# Nome dos arquivos para capital
file_str_earning_cap = 'Basico_' + 'SP1' # renda
file_str_pop_cap = 'pessoa13_' + 'sp1' # população

# Nome dos arquivos para os demais municípios
file_str_earning = 'Basico_' + 'SP2'
file_str_pop = 'pessoa13_' + 'sp2'

In [108]:
# Obtendo os caminhos para os arquivos

# Filtros para os caminhos 
get_path_earning_cap = files_list_df['filepaths'].str.contains(file_str_earning_cap)
get_path_pop_cap = files_list_df['filepaths'].str.contains(file_str_pop_cap)

get_path_earning = files_list_df['filepaths'].str.contains(file_str_earning)
get_path_pop = files_list_df['filepaths'].str.contains(file_str_pop)

# Caminhos para renda
file_path_earning_cap = files_list_df.loc[get_path_earning_cap, 'filepaths'].values[0]
file_path_earning = files_list_df.loc[get_path_earning, 'filepaths'].values[0]
# Caminhos para população
file_path_pop_cap = files_list_df.loc[get_path_pop_cap,'filepaths'].values[0]
file_path_pop = files_list_df.loc[get_path_pop,'filepaths'].values[0]

print(file_path_earning_cap)
print(file_path_pop_cap)
print(file_path_earning)
print(file_path_pop)

c:\Users\guicr\OneDrive\Carreira\Portfolio\uber\data\SP_Capital_20231030\Base informaçoes setores2010 universo SP_Capital\CSV\Basico_SP1.csv
c:\Users\guicr\OneDrive\Carreira\Portfolio\uber\data\SP_Capital_20231030\Base informaçoes setores2010 universo SP_Capital\CSV\pessoa13_sp1.csv
c:\Users\guicr\OneDrive\Carreira\Portfolio\uber\data\SP_Exceto_Capital_20231030\Base informaçoes setores2010 universo SP_Exceto_Capital\CSV\Basico_SP2.csv
c:\Users\guicr\OneDrive\Carreira\Portfolio\uber\data\SP_Exceto_Capital_20231030\Base informaçoes setores2010 universo SP_Exceto_Capital\CSV\pessoa13_sp2.csv


In [109]:
# Passando os arquivos de renda .csv para dataframe
city_earning_cap = pd.read_csv(file_path_earning_cap, encoding='iso-8859-1', sep=';')
city_earning = pd.read_csv(file_path_earning, encoding='iso-8859-1', sep=';')

#### Executando algumas verificações nos arquivos...

    Verificação dos dados referentes as rendas:

In [110]:
# Observando o dataframe
city_earning_cap.head().T

,0,1,2,3,4
Cod_setor,355030801000001,355030801000002,355030801000003,355030801000004,355030801000005
Cod_Grandes Regiões,3,3,3,3,3
Nome_Grande_Regiao,Região Sudeste,Região Sudeste,Região Sudeste,Região Sudeste,Região Sudeste
Cod_UF,35,35,35,35,35
Nome_da_UF,São Paulo,São Paulo,São Paulo,São Paulo,São Paulo
Cod_meso,3515,3515,3515,3515,3515
Nome_da_meso,Metropolitana de São Paulo,Metropolitana de São Paulo,Metropolitana de São Paulo,Metropolitana de São Paulo,Metropolitana de São Paulo
Cod_micro,35061,35061,35061,35061,35061
Nome_da_micro,São Paulo,São Paulo,São Paulo,São Paulo,São Paulo
Cod_RM,20,20,20,20,20


In [111]:
# Informações sobre as colunas
city_earning_cap.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18363 entries, 0 to 18362
Data columns (total 33 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   Cod_setor            18363 non-null  int64  
 1   Cod_Grandes Regiões  18363 non-null  int64  
 2   Nome_Grande_Regiao   18363 non-null  object 
 3   Cod_UF               18363 non-null  int64  
 4   Nome_da_UF           18363 non-null  object 
 5   Cod_meso             18363 non-null  int64  
 6   Nome_da_meso         18363 non-null  object 
 7   Cod_micro            18363 non-null  int64  
 8   Nome_da_micro        18363 non-null  object 
 9   Cod_RM               18363 non-null  int64  
 10  Nome_da_RM           18363 non-null  object 
 11  Cod_municipio        18363 non-null  int64  
 12  Nome_do_municipio    18363 non-null  object 
 13  Cod_distrito         18363 non-null  int64  
 14  Nome_do_distrito     18363 non-null  object 
 15  Cod_subdistrito      18363 non-null 

In [112]:
# Quantidade de dados nulos por coluna
city_earning_cap.isnull().sum()

Cod_setor                  0
Cod_Grandes Regiões        0
Nome_Grande_Regiao         0
Cod_UF                     0
Nome_da_UF                 0
Cod_meso                   0
Nome_da_meso               0
Cod_micro                  0
Nome_da_micro              0
Cod_RM                     0
Nome_da_RM                 0
Cod_municipio              0
Nome_do_municipio          0
Cod_distrito               0
Nome_do_distrito           0
Cod_subdistrito            0
Nome_do_subdistrito        0
Cod_bairro                 0
Nome_do_bairro             0
Situacao_setor             0
V001                      30
V002                      30
V003                      30
V004                      30
V005                      30
V006                      30
V007                      39
V008                      39
V009                       0
V010                       0
V011                      11
V012                      11
Unnamed: 32            18363
dtype: int64

In [113]:
city_earning.head().T

,0,1,2,3,4
Cod_setor,350010505000001,350010505000002,350010505000003,350010505000004,350010505000005
Cod_Grandes Regiões,3,3,3,3,3
Nome_Grande_Regiao,Região Sudeste,Região Sudeste,Região Sudeste,Região Sudeste,Região Sudeste
Cod_UF,35,35,35,35,35
Nome_da_UF,São Paulo,São Paulo,São Paulo,São Paulo,São Paulo
Cod_meso,3508,3508,3508,3508,3508
Nome_da_meso,Presidente Prudente,Presidente Prudente,Presidente Prudente,Presidente Prudente,Presidente Prudente
Cod_micro,35035,35035,35035,35035,35035
Nome_da_micro,Adamantina,Adamantina,Adamantina,Adamantina,Adamantina
Cod_RM,0,0,0,0,0


In [114]:
city_earning.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 47733 entries, 0 to 47732
Data columns (total 33 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   Cod_setor            47733 non-null  int64  
 1   Cod_Grandes Regiões  47733 non-null  int64  
 2   Nome_Grande_Regiao   47733 non-null  object 
 3   Cod_UF               47733 non-null  int64  
 4   Nome_da_UF           47733 non-null  object 
 5   Cod_meso             47733 non-null  int64  
 6   Nome_da_meso         47733 non-null  object 
 7   Cod_micro            47733 non-null  int64  
 8   Nome_da_micro        47733 non-null  object 
 9   Cod_RM               47733 non-null  int64  
 10  Nome_da_RM           47733 non-null  object 
 11  Cod_municipio        47733 non-null  int64  
 12  Nome_do_municipio    47733 non-null  object 
 13  Cod_distrito         47733 non-null  int64  
 14  Nome_do_distrito     47733 non-null  object 
 15  Cod_subdistrito      47733 non-null 

In [115]:
city_earning.isnull().sum()

Cod_setor                  0
Cod_Grandes Regiões        0
Nome_Grande_Regiao         0
Cod_UF                     0
Nome_da_UF                 0
Cod_meso                   0
Nome_da_meso               0
Cod_micro                  0
Nome_da_micro              0
Cod_RM                     0
Nome_da_RM                 0
Cod_municipio              0
Nome_do_municipio          0
Cod_distrito               0
Nome_do_distrito           0
Cod_subdistrito            0
Nome_do_subdistrito        0
Cod_bairro                 0
Nome_do_bairro             0
Situacao_setor             0
V001                     254
V002                     254
V003                     254
V004                     254
V005                     254
V006                     254
V007                     290
V008                     290
V009                       1
V010                       1
V011                      78
V012                      78
Unnamed: 32            47733
dtype: int64

In [116]:
null_values=city_earning.isnull().sum()
null_values[null_values>0]

V001             254
V002             254
V003             254
V004             254
V005             254
V006             254
V007             290
V008             290
V009               1
V010               1
V011              78
V012              78
Unnamed: 32    47733
dtype: int64

Nestas primeiras verificações, percebemos que existem alguns valores nulos em algumas colunas, identificação da estruturação do dataframe contendo código do setor censitário, este que será utilizado para análise dos dados e uma coluna com todos os valores nulos chamada 'Unnamed: 32'. Ao verificar a documentação do arquivo, a informação para a renda per capita de um setor censitário é informado na coluna 'V009'. Esta possui um valor nulo para o dataframe referente aos demais municípios.

Neste momento, será necessário extrair somente as informações necessárias para análise do dataframe 'cru'. Assim, julgo que necessário apenas os dados referentes ao setor censitário, nome do município, nome do bairro e valor da renda per capita.

Vamos observar os valores únicos para nome do município e nome do bairro.

In [117]:
city_earning['Nome_do_municipio'].unique()

array(['ADAMANTINA', 'ADOLFO', 'AGUAÍ', 'ÁGUAS DA PRATA',
       'ÁGUAS DE LINDÓIA', 'ÁGUAS DE SANTA BÁRBARA', 'ÁGUAS DE SÃO PEDRO',
       'AGUDOS', 'ALAMBARI', 'ALFREDO MARCONDES', 'ALTAIR', 'ALTINÓPOLIS',
       'ALTO ALEGRE', 'ALUMÍNIO', 'ÁLVARES FLORENCE', 'ÁLVARES MACHADO',
       'ÁLVARO DE CARVALHO', 'ALVINLÂNDIA', 'AMERICANA',
       'AMÉRICO BRASILIENSE', 'AMÉRICO DE CAMPOS', 'AMPARO', 'ANALÂNDIA',
       'ANDRADINA', 'ANGATUBA', 'ANHEMBI', 'ANHUMAS', 'APARECIDA',
       "APARECIDA D'OESTE", 'APIAÍ', 'ARAÇARIGUAMA', 'ARAÇATUBA',
       'ARAÇOIABA DA SERRA', 'ARAMINA', 'ARANDU', 'ARAPEÍ', 'ARARAQUARA',
       'ARARAS', 'ARCO-ÍRIS', 'AREALVA', 'AREIAS', 'AREIÓPOLIS',
       'ARIRANHA', 'ARTUR NOGUEIRA', 'ARUJÁ', 'ASPÁSIA', 'ASSIS',
       'ATIBAIA', 'AURIFLAMA', 'AVAÍ', 'AVANHANDAVA', 'AVARÉ',
       'BADY BASSITT', 'BALBINOS', 'BÁLSAMO', 'BANANAL',
       'BARÃO DE ANTONINA', 'BARBOSA', 'BARIRI', 'BARRA BONITA',
       'BARRA DO CHAPÉU', 'BARRA DO TURVO', 'BARRETOS', 'BARRINHA

In [118]:
city_earning_cap['Nome_do_municipio'].unique()

array(['SÃO PAULO'], dtype=object)

Realmente, um dataframe possui apenas a capital e no outro os demais municípios.

##### Verificando os bairros contidos nos municípios:

Faz-se uma pesquisa observando os distritos, subdistritos e bairros.

In [119]:
for municipio in city_earning['Nome_do_municipio'].unique():
    print(municipio)
    print(city_earning[city_earning['Nome_do_municipio'].str.fullmatch(municipio)]['Nome_do_subdistrito'].unique())
    print(city_earning[city_earning['Nome_do_municipio'].str.fullmatch(municipio)]['Nome_do_distrito'].unique())
    print(city_earning[city_earning['Nome_do_municipio'].str.fullmatch(municipio)]['Nome_do_bairro'].unique())

ADAMANTINA
['ADAMANTINA']
['ADAMANTINA']
['ADAMANTINA (todos os setores)']
ADOLFO
['ADOLFO']
['ADOLFO']
['ADOLFO (todos os setores)']
AGUAÍ
['AGUAÍ']
['AGUAÍ']
['AGUAÍ (todos os setores)']
ÁGUAS DA PRATA
['ÁGUAS DA PRATA' 'SÃO ROQUE DA FARTURA']
['ÁGUAS DA PRATA' 'SÃO ROQUE DA FARTURA']
['ÁGUAS DA PRATA (todos os setores)']
ÁGUAS DE LINDÓIA
['ÁGUAS DE LINDÓIA']
['ÁGUAS DE LINDÓIA']
['ÁGUAS DE LINDÓIA (todos os setores)']
ÁGUAS DE SANTA BÁRBARA
['ÁGUAS DE SANTA BÁRBARA']
['ÁGUAS DE SANTA BÁRBARA']
['ÁGUAS DE SANTA BÁRBARA (todos os setores)']
ÁGUAS DE SÃO PEDRO
['ÁGUAS DE SÃO PEDRO']
['ÁGUAS DE SÃO PEDRO']
['ÁGUAS DE SÃO PEDRO (todos os setores)']
AGUDOS
['AGUDOS' 'DOMÉLIA']
['AGUDOS' 'DOMÉLIA']
['AGUDOS (todos os setores)']
ALAMBARI
['ALAMBARI']
['ALAMBARI']
['ALAMBARI (todos os setores)']
ALFREDO MARCONDES
['ALFREDO MARCONDES']
['ALFREDO MARCONDES']
['ALFREDO MARCONDES (todos os setores)']
ALTAIR
['ALTAIR' 'SUINANA']
['ALTAIR' 'SUINANA']
['ALTAIR (todos os setores)']
ALTINÓPOLIS
['ALT

A pesquisa nos mostra para os demais municipios que a coluna 'Nome_do_bairro' informa os bairros dos municípios.

In [120]:
for municipio in city_earning_cap['Nome_do_municipio'].unique():
    print(municipio)
    print(city_earning_cap[city_earning_cap['Nome_do_municipio'].str.fullmatch(municipio)]['Nome_do_subdistrito'].unique())
    print(city_earning_cap[city_earning_cap['Nome_do_municipio'].str.fullmatch(municipio)]['Nome_do_distrito'].unique())
    print(city_earning_cap[city_earning_cap['Nome_do_municipio'].str.fullmatch(municipio)]['Nome_do_bairro'].unique())

SÃO PAULO
['ÁGUA RASA' 'ALTO DE PINHEIROS' 'ANHANGUERA' 'ARICANDUVA' 'ARTUR ALVIM'
 'BARRA FUNDA' 'BELA VISTA' 'BELÉM' 'BOM RETIRO' 'BRÁS' 'BRASILÂNDIA'
 'BUTANTÃ' 'CACHOEIRINHA' 'CAMBUCI' 'CAMPO BELO' 'CAMPO GRANDE'
 'CAMPO LIMPO' 'CANGAIBA' 'CAPÃO REDONDO' 'CARRÃO' 'CASA VERDE'
 'CIDADE ADEMAR' 'CIDADE DUTRA' 'CIDADE LIDER' 'CIDADE TIRADENTES'
 'CONSOLAÇÃO' 'CURSINO' 'ERMELINO MATARAZZO' 'FREGUESIA DO Ó' 'GRAJAÚ'
 'GUAIANASES' 'MOEMA' 'IGUATEMI' 'IPIRANGA' 'ITAIM BIBI' 'ITAIM PAULISTA'
 'ITAQUERA' 'JABAQUARA' 'JAÇANÃ' 'JAGUARA' 'JAGUARÉ' 'JARAGUÁ'
 'JARDIM ÂNGELA' 'JARDIM HELENA' 'JARDIM PAULISTA' 'JARDIM SÃO LUÍS'
 'JOSÉ BONIFÁCIO' 'LAPA' 'LIBERDADE' 'LIMÃO' 'MANDAQUI' 'MARSILAC' 'MOOCA'
 'MORUMBI' 'PARELHEIROS' 'PARI' 'PARQUE DO CARMO' 'PEDREIRA' 'PENHA'
 'PERDIZES' 'PERUS' 'PINHEIROS' 'PIRITUBA' 'PONTE RASA' 'RAPOSO TAVARES'
 'REPÚBLICA' 'RIO PEQUENO' 'SACOMÃ' 'SANTA CECÍLIA' 'SANTANA'
 'SANTO AMARO' 'SÃO LUCAS' 'SÃO MATEUS' 'SÃO MIGUEL' 'SÃO RAFAEL'
 'SAPOPEMBA' 'SAÚDE' 'SÉ' 'SOC

No entanto, a capital pode ser obtida pelas colunas 'Nome_do_distrito' ou 'Nome_do_subdistrito'.

Portanto, iremos extrair do dataframe 'cru' apenas as informações das colunas 'Cod_setor','Nome_do_municipio','Nome_do_distrito','Nome_do_bairro' e 'V009'.

In [121]:
stg_city_earning_cap = city_earning_cap[['Cod_setor','Nome_do_municipio','Nome_do_distrito','Nome_do_bairro', 'V009']].copy()
stg_city_earning = city_earning[['Cod_setor','Nome_do_municipio','Nome_do_distrito','Nome_do_bairro', 'V009']].copy()

In [122]:
stg_city_earning_cap.head().T

,0,1,2,3,4
Cod_setor,355030801000001,355030801000002,355030801000003,355030801000004,355030801000005
Nome_do_municipio,SÃO PAULO,SÃO PAULO,SÃO PAULO,SÃO PAULO,SÃO PAULO
Nome_do_distrito,ÁGUA RASA,ÁGUA RASA,ÁGUA RASA,ÁGUA RASA,ÁGUA RASA
Nome_do_bairro,SÃO PAULO (todos os setores),SÃO PAULO (todos os setores),SÃO PAULO (todos os setores),SÃO PAULO (todos os setores),SÃO PAULO (todos os setores)
V009,"1227,41","1045,78","902,97","869,09","917,71"


In [123]:
stg_city_earning.head().T

,0,1,2,3,4
Cod_setor,350010505000001,350010505000002,350010505000003,350010505000004,350010505000005
Nome_do_municipio,ADAMANTINA,ADAMANTINA,ADAMANTINA,ADAMANTINA,ADAMANTINA
Nome_do_distrito,ADAMANTINA,ADAMANTINA,ADAMANTINA,ADAMANTINA,ADAMANTINA
Nome_do_bairro,ADAMANTINA (todos os setores),ADAMANTINA (todos os setores),ADAMANTINA (todos os setores),ADAMANTINA (todos os setores),ADAMANTINA (todos os setores)
V009,"1530,51","1180,37","1908,64","1583,46","1423,52"


In [124]:
# União dos dataframes
stg_city_earning_all = pd.concat([stg_city_earning_cap, stg_city_earning], ignore_index=True)

In [125]:
len(stg_city_earning_cap)+len(stg_city_earning)

66096

In [126]:
stg_city_earning_all.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 66096 entries, 0 to 66095
Data columns (total 5 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   Cod_setor          66096 non-null  int64 
 1   Nome_do_municipio  66096 non-null  object
 2   Nome_do_distrito   66096 non-null  object
 3   Nome_do_bairro     66096 non-null  object
 4   V009               66095 non-null  object
dtypes: int64(1), object(4)
memory usage: 2.5+ MB


Repetição dos passos anteriores para a população.

In [127]:
city_pop_cap = pd.read_csv(file_path_pop_cap, encoding='iso-8859-1', sep=';')
city_pop = pd.read_csv(file_path_pop, encoding='iso-8859-1', sep=',')

In [128]:
city_pop_cap.head().T

,0,1,2,3,4
Cod_setor,355030801000001,355030801000002,355030801000003,355030801000004,355030801000005
Situacao_setor,1,1,1,1,1
V001,806,913,625,572,754
V002,800,913,625,572,754
V003,268,306,189,181,240
...,...,...,...,...,...
V130,1,0,0,0,0
V131,0,0,0,0,0
V132,1,0,0,0,0
V133,0,0,0,0,0


In [129]:
city_pop.head().T

,0,1,2,3,4
Cod_setor,350010505000001,350010505000002,350010505000003,350010505000004,350010505000005
Situacao_setor,1,1,1,1,1
V001,304,785,524,795,813
V002,304,785,519,788,813
V003,149,294,215,288,291
...,...,...,...,...,...
V130,0,0,0,1,1
V131,0,0,0,0,0
V132,0,0,1,0,0
V133,0,0,0,1,0


In [130]:
city_pop_cap.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18363 entries, 0 to 18362
Columns: 136 entries, Cod_setor to V134
dtypes: int64(3), object(133)
memory usage: 19.1+ MB


In [131]:
city_pop.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 47733 entries, 0 to 47732
Columns: 136 entries, Cod_setor to V134
dtypes: int64(3), object(133)
memory usage: 49.5+ MB


In [132]:
null_values = city_pop_cap.isnull().sum()
null_values[null_values > 0]

Series([], dtype: int64)

In [133]:
null_values = city_pop.isnull().sum()
null_values[null_values > 0]

Series([], dtype: int64)

De acordo com a documentação, a coluna 'V001' possui os dados da população por setor censitário. Portanto, iremos extrair esta coluna e a 'Cod_setor' para mesclar os dois dataframes de renda e população.

In [134]:
stg_city_pop_cap = city_pop_cap[['Cod_setor','V001']].copy()
stg_city_pop = city_pop[['Cod_setor','V001']].copy()

In [135]:
# União dos dataframes

stg_city_pop_all = pd.concat([stg_city_pop_cap, stg_city_pop], ignore_index=True)

In [136]:
stg_city_pop_all.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 66096 entries, 0 to 66095
Data columns (total 2 columns):
 #   Column     Non-Null Count  Dtype
---  ------     --------------  -----
 0   Cod_setor  66096 non-null  int64
 1   V001       66096 non-null  int64
dtypes: int64(2)
memory usage: 1.0 MB


##### Mesclagem dos dataframes

In [137]:
# Mudando o tipo de dado int para string

stg_city_pop_all['Cod_setor'] = stg_city_pop_all['Cod_setor'].astype('str')
stg_city_earning_all['Cod_setor'] = stg_city_earning_all['Cod_setor'].astype('str')

Vamos verificar se todos os códigos censitários têm correspondência em ambos os dataframes.

In [138]:
stg_city_earning_all[~stg_city_earning_all['Cod_setor'].isin(stg_city_pop_all['Cod_setor'])]

,Cod_setor,Nome_do_municipio,Nome_do_distrito,Nome_do_bairro,V009


Como o retorno foi nulo, sabe-se que todos os setores censitários possuem correspondência em ambos dataframes.

In [139]:
# Mesclando

stg_city_all = pd.merge(stg_city_earning_all, stg_city_pop_all, on='Cod_setor')

##### Verificando os dados...

In [140]:
stg_city_all.head()

,Cod_setor,Nome_do_municipio,Nome_do_distrito,Nome_do_bairro,V009,V001
0,355030801000001,SÃO PAULO,ÁGUA RASA,SÃO PAULO (todos os setores),"1227,41",806
1,355030801000002,SÃO PAULO,ÁGUA RASA,SÃO PAULO (todos os setores),"1045,78",913
2,355030801000003,SÃO PAULO,ÁGUA RASA,SÃO PAULO (todos os setores),"902,97",625
3,355030801000004,SÃO PAULO,ÁGUA RASA,SÃO PAULO (todos os setores),"869,09",572
4,355030801000005,SÃO PAULO,ÁGUA RASA,SÃO PAULO (todos os setores),"917,71",754


In [141]:
stg_city_all.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 66096 entries, 0 to 66095
Data columns (total 6 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   Cod_setor          66096 non-null  object
 1   Nome_do_municipio  66096 non-null  object
 2   Nome_do_distrito   66096 non-null  object
 3   Nome_do_bairro     66096 non-null  object
 4   V009               66095 non-null  object
 5   V001               66096 non-null  int64 
dtypes: int64(1), object(5)
memory usage: 3.0+ MB


In [142]:
null_values = stg_city_all.isnull().sum()
null_values[null_values > 0]

V009    1
dtype: int64

In [143]:
stg_city_all[stg_city_all['V009'].isnull()]

,Cod_setor,Nome_do_municipio,Nome_do_distrito,Nome_do_bairro,V009,V001
22280,350600305000050,BAURU,BAURU,BAURU (todos os setores),NaN,21


Devido ao dado faltante, exlui-se este dado e utilizaremos um novo dataframe limpo.

In [144]:
stg1_city_all = stg_city_all.dropna().copy()

In [145]:
null_values = stg1_city_all.isnull().sum()
null_values[null_values > 0]

Series([], dtype: int64)

A coluna 'V009' está como object e o dado é um número, então passaremos ela para float.

In [146]:
stg1_city_all['V009'] = stg1_city_all['V009'].str.replace(",", ".")

In [147]:
stg1_city_all['V009'] = stg1_city_all['V009'].astype('float64')

Renomeando as colunas para mais fácil entendimento.

In [148]:
stg1_city_all.rename(columns={'V001':'pop', 'V009':'renda'}, inplace=True)

In [149]:
stg1_city_all.head()

,Cod_setor,Nome_do_municipio,Nome_do_distrito,Nome_do_bairro,renda,pop
0,355030801000001,SÃO PAULO,ÁGUA RASA,SÃO PAULO (todos os setores),1227.41,806
1,355030801000002,SÃO PAULO,ÁGUA RASA,SÃO PAULO (todos os setores),1045.78,913
2,355030801000003,SÃO PAULO,ÁGUA RASA,SÃO PAULO (todos os setores),902.97,625
3,355030801000004,SÃO PAULO,ÁGUA RASA,SÃO PAULO (todos os setores),869.09,572
4,355030801000005,SÃO PAULO,ÁGUA RASA,SÃO PAULO (todos os setores),917.71,754


In [150]:
stg1_city_all.info()

<class 'pandas.core.frame.DataFrame'>
Index: 66095 entries, 0 to 66095
Data columns (total 6 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Cod_setor          66095 non-null  object 
 1   Nome_do_municipio  66095 non-null  object 
 2   Nome_do_distrito   66095 non-null  object 
 3   Nome_do_bairro     66095 non-null  object 
 4   renda              66095 non-null  float64
 5   pop                66095 non-null  int64  
dtypes: float64(1), int64(1), object(4)
memory usage: 3.5+ MB


Como verificado anteriormente, os bairros do município de São Paulo estão na coluna 'Nome_do_distrito'. Portanto, iremos passar as informações desta coluna para a coluna 'Nome_do_bairro'.

In [151]:
stg1_city_all.head(10)

,Cod_setor,Nome_do_municipio,Nome_do_distrito,Nome_do_bairro,renda,pop
0,355030801000001,SÃO PAULO,ÁGUA RASA,SÃO PAULO (todos os setores),1227.41,806
1,355030801000002,SÃO PAULO,ÁGUA RASA,SÃO PAULO (todos os setores),1045.78,913
2,355030801000003,SÃO PAULO,ÁGUA RASA,SÃO PAULO (todos os setores),902.97,625
3,355030801000004,SÃO PAULO,ÁGUA RASA,SÃO PAULO (todos os setores),869.09,572
4,355030801000005,SÃO PAULO,ÁGUA RASA,SÃO PAULO (todos os setores),917.71,754
5,355030801000006,SÃO PAULO,ÁGUA RASA,SÃO PAULO (todos os setores),878.94,643
6,355030801000007,SÃO PAULO,ÁGUA RASA,SÃO PAULO (todos os setores),717.13,690
7,355030801000008,SÃO PAULO,ÁGUA RASA,SÃO PAULO (todos os setores),686.94,755
8,355030801000009,SÃO PAULO,ÁGUA RASA,SÃO PAULO (todos os setores),1006.66,693
9,355030801000010,SÃO PAULO,ÁGUA RASA,SÃO PAULO (todos os setores),1293.43,676


In [152]:
# Transferindo os dados do distrito para bairro.
stg1_city_all.loc[stg1_city_all['Nome_do_municipio'].str.fullmatch('SÃO PAULO'), 'Nome_do_bairro'] = stg1_city_all['Nome_do_distrito']

Primeira análise para verificar se foram transferidos os dados para 'Nome_do_bairro'.

In [153]:
stg1_city_all.head(10)

,Cod_setor,Nome_do_municipio,Nome_do_distrito,Nome_do_bairro,renda,pop
0,355030801000001,SÃO PAULO,ÁGUA RASA,ÁGUA RASA,1227.41,806
1,355030801000002,SÃO PAULO,ÁGUA RASA,ÁGUA RASA,1045.78,913
2,355030801000003,SÃO PAULO,ÁGUA RASA,ÁGUA RASA,902.97,625
3,355030801000004,SÃO PAULO,ÁGUA RASA,ÁGUA RASA,869.09,572
4,355030801000005,SÃO PAULO,ÁGUA RASA,ÁGUA RASA,917.71,754
5,355030801000006,SÃO PAULO,ÁGUA RASA,ÁGUA RASA,878.94,643
6,355030801000007,SÃO PAULO,ÁGUA RASA,ÁGUA RASA,717.13,690
7,355030801000008,SÃO PAULO,ÁGUA RASA,ÁGUA RASA,686.94,755
8,355030801000009,SÃO PAULO,ÁGUA RASA,ÁGUA RASA,1006.66,693
9,355030801000010,SÃO PAULO,ÁGUA RASA,ÁGUA RASA,1293.43,676


Seegunda análise, verificando se todos os dados permanecem inalterados.

In [154]:
for municipio in stg1_city_all['Nome_do_municipio'].unique():
    print(municipio)
    print(stg1_city_all[stg1_city_all['Nome_do_municipio'].str.fullmatch(municipio)]['Nome_do_bairro'].unique())

SÃO PAULO
['ÁGUA RASA' 'ALTO DE PINHEIROS' 'ANHANGUERA' 'ARICANDUVA' 'ARTUR ALVIM'
 'BARRA FUNDA' 'BELA VISTA' 'BELÉM' 'BOM RETIRO' 'BRÁS' 'BRASILÂNDIA'
 'BUTANTÃ' 'CACHOEIRINHA' 'CAMBUCI' 'CAMPO BELO' 'CAMPO GRANDE'
 'CAMPO LIMPO' 'CANGAIBA' 'CAPÃO REDONDO' 'CARRÃO' 'CASA VERDE'
 'CIDADE ADEMAR' 'CIDADE DUTRA' 'CIDADE LIDER' 'CIDADE TIRADENTES'
 'CONSOLAÇÃO' 'CURSINO' 'ERMELINO MATARAZZO' 'FREGUESIA DO Ó' 'GRAJAÚ'
 'GUAIANASES' 'MOEMA' 'IGUATEMI' 'IPIRANGA' 'ITAIM BIBI' 'ITAIM PAULISTA'
 'ITAQUERA' 'JABAQUARA' 'JAÇANÃ' 'JAGUARA' 'JAGUARÉ' 'JARAGUÁ'
 'JARDIM ÂNGELA' 'JARDIM HELENA' 'JARDIM PAULISTA' 'JARDIM SÃO LUÍS'
 'JOSÉ BONIFÁCIO' 'LAPA' 'LIBERDADE' 'LIMÃO' 'MANDAQUI' 'MARSILAC' 'MOOCA'
 'MORUMBI' 'PARELHEIROS' 'PARI' 'PARQUE DO CARMO' 'PEDREIRA' 'PENHA'
 'PERDIZES' 'PERUS' 'PINHEIROS' 'PIRITUBA' 'PONTE RASA' 'RAPOSO TAVARES'
 'REPÚBLICA' 'RIO PEQUENO' 'SACOMÃ' 'SANTA CECÍLIA' 'SANTANA'
 'SANTO AMARO' 'SÃO LUCAS' 'SÃO MATEUS' 'SÃO MIGUEL' 'SÃO RAFAEL'
 'SAPOPEMBA' 'SAÚDE' 'SÉ' 'SOC

Após essa análise, percebemos que todos os municípios estão com os seus bairros na coluna 'Nome_do_bairro'. Pode-se prosseguir para os dados geográficos.

---

## Tratamento dos dados geográficos

Nesta situação, os dados provenientes do arquivo shapefile(.shp) informarão os dados geográficos para localização dos setores e cálculos de área de cada setor censitário. O arquivo para o estado de São Paulo é o 35SEE250GC_SIR.shp.

In [155]:
# Localizando o arquivo
file_str_geo = '35SEE250GC_SIR.shp'
get_path_geo = files_list_df['filepaths'].str.contains(file_str_geo)
file_path_geo = files_list_df.loc[get_path_geo, 'filepaths'].values[0]
print(file_path_geo)

c:\Users\guicr\OneDrive\Carreira\Portfolio\uber\data\sp_setores_censitarios\35SEE250GC_SIR.shp


Utiliza-se a biblioteca geopandas para tratamento dos dados geográficos.

In [156]:
cities_setores_censitarios = gpd.read_file(file_path_geo)

In [157]:
cities_setores_censitarios.head().T

,0,1,2,3,4
ID,98237.0,98232.0,98230.0,98229.0,98231.0
CD_GEOCODI,354100005000009,354100005000004,354100005000002,354100005000001,354100005000003
TIPO,URBANO,URBANO,URBANO,URBANO,URBANO
CD_GEOCODS,35410000500,35410000500,35410000500,35410000500,35410000500
NM_SUBDIST,NaN,NaN,NaN,NaN,NaN
CD_GEOCODD,354100005,354100005,354100005,354100005,354100005
NM_DISTRIT,PRAIA GRANDE,PRAIA GRANDE,PRAIA GRANDE,PRAIA GRANDE,PRAIA GRANDE
CD_GEOCODM,3541000,3541000,3541000,3541000,3541000
NM_MUNICIP,PRAIA GRANDE,PRAIA GRANDE,PRAIA GRANDE,PRAIA GRANDE,PRAIA GRANDE
NM_MICRO,SANTOS,SANTOS,SANTOS,SANTOS,SANTOS


In [158]:
# cópia do arquivo 'cru'
stg_cities_setores_censitarios = cities_setores_censitarios.copy()

##### Verificando os dados...

In [159]:
stg_cities_setores_censitarios.head(10).T

,0,1,2,3,4,5,6,7,8,9
ID,98237.0,98232.0,98230.0,98229.0,98231.0,98233.0,98236.0,98242.0,98239.0,98238.0
CD_GEOCODI,354100005000009,354100005000004,354100005000002,354100005000001,354100005000003,354100005000005,354100005000008,354100005000014,354100005000011,354100005000010
TIPO,URBANO,URBANO,URBANO,URBANO,URBANO,URBANO,URBANO,URBANO,URBANO,URBANO
CD_GEOCODS,35410000500,35410000500,35410000500,35410000500,35410000500,35410000500,35410000500,35410000500,35410000500,35410000500
NM_SUBDIST,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
CD_GEOCODD,354100005,354100005,354100005,354100005,354100005,354100005,354100005,354100005,354100005,354100005
NM_DISTRIT,PRAIA GRANDE,PRAIA GRANDE,PRAIA GRANDE,PRAIA GRANDE,PRAIA GRANDE,PRAIA GRANDE,PRAIA GRANDE,PRAIA GRANDE,PRAIA GRANDE,PRAIA GRANDE
CD_GEOCODM,3541000,3541000,3541000,3541000,3541000,3541000,3541000,3541000,3541000,3541000
NM_MUNICIP,PRAIA GRANDE,PRAIA GRANDE,PRAIA GRANDE,PRAIA GRANDE,PRAIA GRANDE,PRAIA GRANDE,PRAIA GRANDE,PRAIA GRANDE,PRAIA GRANDE,PRAIA GRANDE
NM_MICRO,SANTOS,SANTOS,SANTOS,SANTOS,SANTOS,SANTOS,SANTOS,SANTOS,SANTOS,SANTOS


Os dados geográficos apresentam as seguintes relevantes para essa análise:
1. CD_GEOCODI - Este é código do setor censitário para referenciarmos ao censo 2010;
2. geometry - Dados do polígono que contem o censo censitário para cálculo da área do mesmo.

In [160]:
stg_cities_setores_censitarios.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
RangeIndex: 68296 entries, 0 to 68295
Data columns (total 15 columns):
 #   Column      Non-Null Count  Dtype   
---  ------      --------------  -----   
 0   ID          68296 non-null  float64 
 1   CD_GEOCODI  68296 non-null  object  
 2   TIPO        68296 non-null  object  
 3   CD_GEOCODS  68296 non-null  object  
 4   NM_SUBDIST  0 non-null      float64 
 5   CD_GEOCODD  68296 non-null  object  
 6   NM_DISTRIT  68296 non-null  object  
 7   CD_GEOCODM  68296 non-null  object  
 8   NM_MUNICIP  68296 non-null  object  
 9   NM_MICRO    68296 non-null  object  
 10  NM_MESO     68296 non-null  object  
 11  CD_GEOCODB  18797 non-null  object  
 12  NM_BAIRRO   18797 non-null  object  
 13  ID1         68296 non-null  int64   
 14  geometry    68296 non-null  geometry
dtypes: float64(2), geometry(1), int64(1), object(11)
memory usage: 7.8+ MB


In [161]:
stg_cities_setores_censitarios.isnull().sum()

ID                0
CD_GEOCODI        0
TIPO              0
CD_GEOCODS        0
NM_SUBDIST    68296
CD_GEOCODD        0
NM_DISTRIT        0
CD_GEOCODM        0
NM_MUNICIP        0
NM_MICRO          0
NM_MESO           0
CD_GEOCODB    49499
NM_BAIRRO     49499
ID1               0
geometry          0
dtype: int64

Ao analisar as informações percebe-se que todos os dados necessários estão completos sem valores faltantes.

In [162]:
# Renomeando a coluna 'CD_GEOCODI' para 'Cod_setor' a fim de criar a mesma referência em ambos os dataframes
stg_cities_setores_censitarios.rename(columns={'CD_GEOCODI':'Cod_setor'}, inplace=True)
stg_cities_setores_censitarios.head().T

,0,1,2,3,4
ID,98237.0,98232.0,98230.0,98229.0,98231.0
Cod_setor,354100005000009,354100005000004,354100005000002,354100005000001,354100005000003
TIPO,URBANO,URBANO,URBANO,URBANO,URBANO
CD_GEOCODS,35410000500,35410000500,35410000500,35410000500,35410000500
NM_SUBDIST,NaN,NaN,NaN,NaN,NaN
CD_GEOCODD,354100005,354100005,354100005,354100005,354100005
NM_DISTRIT,PRAIA GRANDE,PRAIA GRANDE,PRAIA GRANDE,PRAIA GRANDE,PRAIA GRANDE
CD_GEOCODM,3541000,3541000,3541000,3541000,3541000
NM_MUNICIP,PRAIA GRANDE,PRAIA GRANDE,PRAIA GRANDE,PRAIA GRANDE,PRAIA GRANDE
NM_MICRO,SANTOS,SANTOS,SANTOS,SANTOS,SANTOS


O cálculo de área é realizado na projeção UTM, portanto será necessário realizar a mudança de coordenada geográfica para cartesiana.

In [163]:
# Mudança de coordenada
stg_cities_setores_censitarios = stg_cities_setores_censitarios.to_crs(epsg=31983) # Coordenada UTM para São Paulo é a EPSG 31983
stg_cities_setores_censitarios['geometry'].head()

0    POLYGON ((356565.548 7343834.128, 356417.318 7...
1    POLYGON ((355964.368 7344778.533, 355971.068 7...
2    POLYGON ((356369.535 7344176.257, 356337.978 7...
3    POLYGON ((356447.233 7344442.942, 356383.829 7...
4    POLYGON ((356251.286 7344483.503, 356090.929 7...
Name: geometry, dtype: geometry

In [164]:
# Cálculo da área para hectares
stg_cities_setores_censitarios['area_ha'] = stg_cities_setores_censitarios['geometry'].area / 1e4

O retorno para a coordenada geográfica é necessária para a representação visual em mapas posteriormente.

In [165]:
# Retorno para coordenada geográfica
stg_cities_setores_censitarios = stg_cities_setores_censitarios.to_crs(epsg=4326)

In [166]:
stg_cities_setores_censitarios.head().T

,0,1,2,3,4
ID,98237.0,98232.0,98230.0,98229.0,98231.0
Cod_setor,354100005000009,354100005000004,354100005000002,354100005000001,354100005000003
TIPO,URBANO,URBANO,URBANO,URBANO,URBANO
CD_GEOCODS,35410000500,35410000500,35410000500,35410000500,35410000500
NM_SUBDIST,NaN,NaN,NaN,NaN,NaN
CD_GEOCODD,354100005,354100005,354100005,354100005,354100005
NM_DISTRIT,PRAIA GRANDE,PRAIA GRANDE,PRAIA GRANDE,PRAIA GRANDE,PRAIA GRANDE
CD_GEOCODM,3541000,3541000,3541000,3541000,3541000
NM_MUNICIP,PRAIA GRANDE,PRAIA GRANDE,PRAIA GRANDE,PRAIA GRANDE,PRAIA GRANDE
NM_MICRO,SANTOS,SANTOS,SANTOS,SANTOS,SANTOS


In [ ]:
# Transformando a coluna 'Cod_setor' para string para posterior mesclagem dos dataframes 
stg_cities_setores_censitarios['Cod_setor'] = stg_cities_setores_censitarios['Cod_setor'].astype('str')

---

## EDA - Análise exploratória de dados

In [86]:
len(stg_cities_setores_censitarios['Cod_setor'])

68296

In [87]:
len(stg1_city_all['Cod_setor'])

66095

Verifica-se que existem maior quantidade de setores censitários no dataframe geográfico do que no dataframe dos dados do censo. Isso decorre da legislação brasileira, que exige que as instituições responsáveis por realizar pesquisas estatísticas forneçam informações ao IBGE e protejam os dados de forma a garantir a não identificação dos informantes. Isso significa que, para proteger os dados, o IBGE pode usar medidas como restrição de dados (reduzir o volume de informação liberada) ou restrição de acesso (estabelecer condições para o uso dos dados). No caso do Censo Demográfico 2010, o IBGE optou por restrição de dados e omitiu os valores das maioria das variáveis em setores com menos de cinco domicílios particulares permanentes, conforme a documentação.

Realiza-se a mesclagem dos dados tendo como base os dados do censo para continuação do EDA.

In [169]:
# Copiando apenas os dados relevantes para o EDA
stg1_cities_setores_censitarios = stg_cities_setores_censitarios[['Cod_setor', 'geometry', 'area_ha']].copy()

In [170]:
stg1_cities_setores_censitarios.head().T

,0,1,2,3,4
Cod_setor,354100005000009,354100005000004,354100005000002,354100005000001,354100005000003
geometry,"POLYGON ((-46.410239 -24.011028, -46.411695 -2...","POLYGON ((-46.416055 -24.002445999999996, -46....","POLYGON ((-46.412132 -24.007920999999996, -46....","POLYGON ((-46.411342 -24.005519999999997, -46....","POLYGON ((-46.413264 -24.005136, -46.414837999..."
area_ha,2.382885,10.134651,3.067219,8.440376,4.89736


In [171]:
# Mesclando todos os dados, nesta  mesclagem fica-se apenas com os dados do censo
stg_cities = pd.merge(stg1_city_all, stg1_cities_setores_censitarios, on='Cod_setor')

In [172]:
stg_cities.head().T

,0,1,2,3,4
Cod_setor,355030801000001,355030801000002,355030801000003,355030801000004,355030801000005
Nome_do_municipio,SÃO PAULO,SÃO PAULO,SÃO PAULO,SÃO PAULO,SÃO PAULO
Nome_do_distrito,ÁGUA RASA,ÁGUA RASA,ÁGUA RASA,ÁGUA RASA,ÁGUA RASA
Nome_do_bairro,ÁGUA RASA,ÁGUA RASA,ÁGUA RASA,ÁGUA RASA,ÁGUA RASA
renda,1227.41,1045.78,902.97,869.09,917.71
pop,806,913,625,572,754
geometry,"POLYGON ((-46.569514 -23.569162999999996, -46....","POLYGON ((-46.568704 -23.568531999999994, -46....","POLYGON ((-46.568557 -23.56965199999999, -46.5...",POLYGON ((-46.56935668536443 -23.5730222216808...,"POLYGON ((-46.572295 -23.569816, -46.569494999..."
area_ha,7.213308,7.162393,5.596828,6.840022,8.228113


In [173]:
stg_cities.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 66095 entries, 0 to 66094
Data columns (total 8 columns):
 #   Column             Non-Null Count  Dtype   
---  ------             --------------  -----   
 0   Cod_setor          66095 non-null  object  
 1   Nome_do_municipio  66095 non-null  object  
 2   Nome_do_distrito   66095 non-null  object  
 3   Nome_do_bairro     66095 non-null  object  
 4   renda              66095 non-null  float64 
 5   pop                66095 non-null  int64   
 6   geometry           66095 non-null  geometry
 7   area_ha            66095 non-null  float64 
dtypes: float64(2), geometry(1), int64(1), object(4)
memory usage: 4.0+ MB


In [174]:
stg_cities.isnull().sum()

Cod_setor            0
Nome_do_municipio    0
Nome_do_distrito     0
Nome_do_bairro       0
renda                0
pop                  0
geometry             0
area_ha              0
dtype: int64

Todas as informações foram mescladas.

Cálculo da densidade demográfica como um índice de aglomeração para entendimento do adensamento populacional.

In [175]:
stg_cities['dens_pop'] = stg_cities['pop'] / stg_cities['area_ha']

In [176]:
stg_cities.head().T

,0,1,2,3,4
Cod_setor,355030801000001,355030801000002,355030801000003,355030801000004,355030801000005
Nome_do_municipio,SÃO PAULO,SÃO PAULO,SÃO PAULO,SÃO PAULO,SÃO PAULO
Nome_do_distrito,ÁGUA RASA,ÁGUA RASA,ÁGUA RASA,ÁGUA RASA,ÁGUA RASA
Nome_do_bairro,ÁGUA RASA,ÁGUA RASA,ÁGUA RASA,ÁGUA RASA,ÁGUA RASA
renda,1227.41,1045.78,902.97,869.09,917.71
pop,806,913,625,572,754
geometry,"POLYGON ((-46.569514 -23.569162999999996, -46....","POLYGON ((-46.568704 -23.568531999999994, -46....","POLYGON ((-46.568557 -23.56965199999999, -46.5...",POLYGON ((-46.56935668536443 -23.5730222216808...,"POLYGON ((-46.572295 -23.569816, -46.569494999..."
area_ha,7.213308,7.162393,5.596828,6.840022,8.228113
dens_pop,111.737912,127.471356,111.670402,83.625464,91.637055


Neste momento, analisa-se algumas informações relevantes sobre os municípios. Os dois aspectos a serem observados serão a renda e densidade populacional para cumprimento do objetivo desta análise.

In [179]:
municipios_gb =  stg_cities.groupby('Nome_do_municipio').agg({'renda':'mean', 'pop':'sum', 'area_ha':'sum'})
municipios_gb['dens_pop'] = municipios_gb['pop'] / municipios_gb['area_ha']

In [183]:
dens_mun = municipios_gb.sort_values(by='dens_pop', ascending=False)
dens_mun

,renda,pop,area_ha,dens_pop
Nome_do_municipio,,,,
DIADEMA,643.421240,386089,3066.708797,125.896857
TABOÃO DA SERRA,799.197833,244528,2035.516803,120.130671
CARAPICUÍBA,686.338806,369584,3409.882837,108.386129
OSASCO,856.330819,666740,6254.383093,106.603639
SÃO CAETANO DO SUL,1804.245680,149263,1532.213100,97.416606
...,...,...,...,...
NOVA CASTILHO,750.062500,1125,18449.353134,0.060978
SANTO ANTÔNIO DO ARACANGUÁ,682.080294,7626,131742.639664,0.057886
LUTÉCIA,746.036111,2714,47816.994778,0.056758


In [185]:
renda_mun = municipios_gb.sort_values(by='renda', ascending=False)
renda_mun

,renda,pop,area_ha,dens_pop
Nome_do_municipio,,,,
SANTANA DE PARNAÍBA,2272.820654,108813,17985.129499,6.050165
SÃO CAETANO DO SUL,1804.245680,149263,1532.213100,97.416606
VINHEDO,1589.086444,63611,8162.151679,7.793411
ÁGUAS DE SÃO PEDRO,1536.731429,2707,554.504265,4.881838
SANTOS,1526.836290,419400,25028.913445,16.756620
...,...,...,...,...
BARRA DO TURVO,343.673571,7729,89171.363338,0.086676
RIBEIRÃO BRANCO,335.451316,18269,69948.203136,0.261179
IPORANGA,320.700000,4299,109826.459079,0.039144


Quais são os top 10?

In [186]:
dens_mun[:10]

,renda,pop,area_ha,dens_pop
Nome_do_municipio,,,,
DIADEMA,643.421240,386089,3066.708797,125.896857
TABOÃO DA SERRA,799.197833,244528,2035.516803,120.130671
CARAPICUÍBA,686.338806,369584,3409.882837,108.386129
OSASCO,856.330819,666740,6254.383093,106.603639
SÃO CAETANO DO SUL,1804.245680,149263,1532.213100,97.416606
SÃO PAULO,1413.447912,11253503,138214.534706,81.420547
MAUÁ,685.262520,417064,5764.162181,72.354661
JANDIRA,835.422222,108344,1745.273844,62.078510
POÁ,671.977051,106013,1725.973133,61.422161


In [187]:
renda_mun[:10]

,renda,pop,area_ha,dens_pop
Nome_do_municipio,,,,
SANTANA DE PARNAÍBA,2272.820654,108813,17985.129499,6.050165
SÃO CAETANO DO SUL,1804.245680,149263,1532.213100,97.416606
VINHEDO,1589.086444,63611,8162.151679,7.793411
ÁGUAS DE SÃO PEDRO,1536.731429,2707,554.504265,4.881838
SANTOS,1526.836290,419400,25028.913445,16.756620
VALINHOS,1461.846025,106793,14848.302557,7.192270
SÃO PAULO,1413.447912,11253503,138214.534706,81.420547
JUNDIAÍ,1381.622470,370126,42492.888913,8.710304
CAMPINAS,1369.791509,1080113,78137.255184,13.823278


Assim, conseguimos saber para os municípios de São Paulo quais possuem maior renda e densidade demográfica. No entanto, o que agrega valor ao estudo é saber, por setores censitários de cada município, onde estão as maiores rendas e densidades. 

De acordo com as autoras do artigo "NÍVEIS DE DENSIDADE POPULACIONAL: Uma proposta de classificação para Goiânia – GO, aplicação no Setor Central" (https://xviiienanpur.anpur.org.br/anaisadmin/capapdf.php?reqid=1034) podemos classificar as densidades populacionais como, em habitantes/hectare, " <15 muito baixa, de 16 a 50 baixa, de 51 a 150 média, de 151 a 350 alta e > 350 muito alta."

Além disso, utilizando a informação do artigo do site einvestidor.estadao.com.br ("Quanto ganha a classe A no Brasil?" Quanto ganha a classe A no Brasil? https://einvestidor.estadao.com.br/radar-einvestidor/quanto-ganha-a-classe-a-no-brasil/) obtém-se os dados referentes as rendas domiciliares por classe social:
- Classe A: R$ 28.240;
- Classe B1: R$ 12.683,34;
- Classe B2: R$ 7.017,64;
- Classe C1: R$ 3.980,38;
- Classe C2: R$ 2.403,04;
- Classe D-E: R$ 1.087,77.

Ademais, o artigo Tendências e Desigualdades da Mobilidade Urbana no Brasil II: Características e Padrões de Consumo da Mobilidade por Aplicativo”, conduzido por Lucas Warwar e Rafael H. M. Pereira (https://repositorio.ipea.gov.br/bitstream/11058/11240/1/td_2781.pdf) demonstra que "A distribuição dos usuários de ride-hailing (viagens por aplicativos) por faixas de renda evidencia uma forte concentração entre os estratos superiores".

De acordo com informações anteriores, consegue-se obter os dados relevantes das condições sócio-demográficas para a descoberta das regiões mais propensas a se obter viagens por aplicativo. Portanto, os dados deverão ser estratificados em regiões com densidade demográfica maior que 150 hab/ha e renda domiciliar acima de R$ 7.017,64. No entanto, os dados tratados, até este momento, não possuem as informações por domícilio, porém os dados do IBGE possuem tal valor. Assim, outro arquivo será criado para tratamento destes dados.

---

### Preparação da visualização a ser utilizada no arquivo atualizado.

Visualização de dados

In [ ]:
plt.figure(figsize=(15,5))

plt.subplot(1,2,1)
plt.hist(stg_city['renda'], bins=30, edgecolor='k')
plt.xlabel('R$ per capita')
plt.ylabel('Frequencia')

plt.subplot(1,2,2)
stg_city.boxplot(column='renda')
plt.ylabel('R$ per capita')

plt.show()

In [ ]:
plt.figure(figsize=(15,5))

plt.subplot(1,2,1)
plt.hist(stg_city['pop'], bins=30, edgecolor='k')
plt.xlabel('População - hab')
plt.ylabel('Frequencia')

plt.subplot(1,2,2)
stg_city.boxplot(column='pop')
plt.ylabel('População')

plt.show()

In [ ]:
plt.figure(figsize=(15,5))

plt.subplot(1,2,1)
plt.hist(stg_city['dens_pop'], bins=30, edgecolor='k')
plt.xlabel('Densidade populacional - hab/ha')
plt.ylabel('Frequencia')

plt.subplot(1,2,2)
stg_city.boxplot(column='dens_pop')
plt.ylabel('Densidade populacional - hab/ha')

plt.show()

In [ ]:
stg_city.describe()

In [ ]:
q3_renda = stg_city['renda'].quantile(0.75)
q3_dens_pop = stg_city['dens_pop'].quantile(0.50)

mask = (stg_city['renda'] >= q3_renda) & (stg_city['dens_pop'] >= q3_dens_pop)
top_25_stg_city = stg_city[mask].copy()
top_25_stg_city.head().T

In [ ]:
top_25_stg_city.info()

In [ ]:
top_25_stg_city.describe()

Visualização geográfica

In [92]:
top_25_stg_city_gdf = gpd.GeoDataFrame(top_25_stg_city, geometry='geometry')

In [ ]:
hotels_list_path = os.path.join(data_dir, 'hotels_list.csv')

hotels_list_df = pd.read_csv(hotels_list_path)

hotels_list_df

In [94]:
map = folium.Map(location=[-23.5633, -46.66744],
                 tiles='Cartodb Positron',
                 zoom_start=13)

borders_style = {
    'color': 'green',
    'weight': 1,
    'fillColor': 'green',
    'fillOpacity': 0.15,
}

top_25 = folium.GeoJson(data=top_25_stg_city_gdf,
                             name='Top 25% renda e densidade populacional',
                             style_function=lambda x: borders_style,
                             )

top_25.add_to(map)

for index, row in hotels_list_df.iterrows():
    folium.Marker(
        location=[row['latitude'], row['longitude']],
        popup=row['name'],
    ).add_to(map)

map_dir = os.path.join(artifacts_dir, "top_25_sao_paulo.html")

map.save(map_dir)